In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

# Any results you write to the current directory are saved as output.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [7]:
books = pd.read_csv('./input/books.csv', encoding = "utf-8").drop_duplicates(subset=['title'])
ratings = pd.read_csv('./input/ratings.csv', encoding = "ISO-8859-1")
book_tags = pd.read_csv('./input/book_tags.csv', encoding = "ISO-8859-1")
tags = pd.read_csv('./input/tags.csv')

tags_join_DF = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
tags_join_DF.head()

to_read = pd.read_csv('./input/to_read.csv')
to_read.head()

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [8]:
books_with_tags = pd.merge(books, tags_join_DF, left_on='book_id', right_on='goodreads_book_id', how='inner')
temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()
books = pd.merge(books, temp_df, left_on='book_id', right_on='book_id', how='outer')
books['corpus'] = (pd.Series(books[['authors', 'tag_name']]
                .fillna('')
                .values.tolist()
                ).str.join(' '))

In [10]:
tf_corpus = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix_corpus = tf_corpus.fit_transform(books['corpus'])
cosine_sim_corpus = linear_kernel(tfidf_matrix_corpus, tfidf_matrix_corpus)
indices1 = pd.Series(books.index, index=books['title'])

# Build a 1-dimensional array with book titles
titles = books['title']
indices = pd.Series(books.index, index=books['title'])

# Function that get book recommendations based on the cosine similarity score of books tags
def corpus_recommendations(title):
    idx = indices1[title]
    sim_scores = list(enumerate(cosine_sim_corpus[idx]))
    if len(sim_scores[0])==2:
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    else:
        sim_scores = sorted(sim_scores, key=lambda x: x[1][0], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

corpus_recommendations("The Hobbit")

18      The Fellowship of the Ring (The Lord of the Ri...
154            The Two Towers (The Lord of the Rings, #2)
160     The Return of the King (The Lord of the Rings,...
4883          Unfinished Tales ofNúmenor and Middle-Earth
2271                                The Children of Húrin
605              The Silmarillion (Middle-Earth Universe)
8097                   The Complete Guide to Middle-Earth
1115     The History of the Hobbit, Part One: Mr. Baggins
461                             The Hobbit: Graphic Novel
35                              The Giver (The Giver, #1)
Name: title, dtype: object

In [23]:
temp = indices["The Hunger Games (The Hunger Games, #1)"]
sim_scores = list(enumerate(cosine_sim_corpus[temp]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
len(sim_scores[0])

2

In [35]:
corpus_recommendations("Twilight (Twilight, #1)")

4                                        The Great Gatsby
0                 The Hunger Games (The Hunger Games, #1)
5                                  The Fault in Our Stars
9                                     Pride and Prejudice
1       Harry Potter and the Sorcerer's Stone (Harry P...
6174                           Private Games (Private #3)
1974                              The Death of Ivan Ilych
374                                          The Namesake
7                                  The Catcher in the Rye
5794                                    Our Mutual Friend
Name: title, dtype: object

In [ ]:
recommendations = [] 
error = []

for index, row in books.iterrows():
    temp = [row['book_id'],row['title']]
    rec =  corpus_recommendations(row['title'])
    for r in rec.index.tolist():
        temp.append(r+1)
    recommendations.append(temp)


In [31]:
recommendations

[[1,
  u'The Hunger Games (The Hunger Games, #1)',
  4,
  2,
  5,
  1,
  9,
  6174,
  1974,
  7,
  374,
  5794],
 [2,
  u"Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
  0,
  5,
  4,
  2,
  9,
  6174,
  7832,
  7819,
  1974,
  7],
 [3, u'Twilight (Twilight, #1)', 4, 0, 5, 9, 1, 6174, 1974, 374, 7, 5794],
 [4,
  u'To Kill a Mockingbird',
  527,
  7095,
  9675,
  6367,
  8218,
  9163,
  472,
  1342,
  1404,
  1579],
 [5, u'The Great Gatsby', 2, 0, 5, 9, 1, 6174, 1974, 7, 374, 5794],
 [6, u'The Fault in Our Stars', 4, 2, 0, 1, 9, 6174, 1974, 7, 5794, 374],
 [7, u'The Hobbit', 18, 154, 160, 4883, 2271, 605, 8097, 1115, 461, 35],
 [8, u'The Catcher in the Rye', 9, 4, 0, 2, 5, 1, 1974, 6174, 29, 7832],
 [9,
  u'Angels & Demons  (Robert Langdon, #1)',
  144,
  169,
  198,
  236,
  722,
  1463,
  3194,
  3195,
  4795,
  5232],
 [10, u'Pride and Prejudice', 0, 4, 2, 5, 1, 7, 1974, 7832, 6174, 7819],
 [11, u'The Kite Runner', 8512, 12, 8511, 361, 8515, 2453, 353, 32, 822, 8722],
 [1

In [33]:
df = pd.DataFrame(recommendations)

df.columns = ['book_id', 'title', '1', '2','3','4','5','6','7','8','9','10']

df

,book_id,title,1,2,3,4,5,6,7,8,9,10
0,1,"The Hunger Games (The Hunger Games, #1)",4,2,5,1,9,6174,1974,7,374,5794
1,2,Harry Potter and the Sorcerer's Stone (Harry P...,0,5,4,2,9,6174,7832,7819,1974,7
2,3,"Twilight (Twilight, #1)",4,0,5,9,1,6174,1974,374,7,5794
3,4,To Kill a Mockingbird,527,7095,9675,6367,8218,9163,472,1342,1404,1579
4,5,The Great Gatsby,2,0,5,9,1,6174,1974,7,374,5794
5,6,The Fault in Our Stars,4,2,0,1,9,6174,1974,7,5794,374
6,7,The Hobbit,18,154,160,4883,2271,605,8097,1115,461,35
7,8,The Catcher in the Rye,9,4,0,2,5,1,1974,6174,29,7832
8,9,"Angels & Demons (Robert Langdon, #1)",144,169,198,236,722,1463,3194,3195,4795,5232
9,10,Pride and Prejudice,0,4,2,5,1,7,1974,7832,6174,7819
